b简答题：
1. 如果你的训练集具有数百万个特征，那么可以使用哪种线性回归训练算法？
随机梯度下降。

2. 如果你的训练集里特征的数值大小迥异，那么哪些算法可能会受到影响？受影响程度如何？你应该怎么做？
梯度下降有关的都会影响（线性回归、逻辑回归等监督学习）
数据标准化或归一化可以解决。

3. 训练逻辑回归模型时，梯度下降可能会卡在局部最小值中吗？
梯度下降保证收敛到全局最小值，所以并不会卡住。

4. 如果你让它们运行足够长的时间，是否所有的梯度下降算法都能得出相同的模型？
不一定。只有数据的学习率比较小，特征比较少才能得出相同的模型。

5. 假设你使用批量梯度下降，并在每个轮次绘制验证误差。如果你发现验证错误持续上升，那么可能是什么情况？你该如何解决？
出现了过拟合。
正则化，增加有用的数据，特征降维。

6. 当验证误差上升时立即停止小批量梯度下降是个好主意吗？
不一定，要观察梯度平滑度在做决定。

7. 哪种梯度下降算法（在我们讨论过的算法中）将最快地到达最佳解附近？哪个实际上会收敛？如何使其他的也收敛
随机梯度下降 和 批量梯度下降，SVM和KNN也可以调参。

8. 假设你正在使用多项式回归。绘制学习曲线后，你会发现训练误差和验证误差之间存在很大的差距。发生了什么？解决此问题的三种方法是什么？
发生了过拟合。多项式回归模型可能过于复杂，导致模型学习了训练数据中的噪声。
正则化，减少阶数，增加训练数

9. 假设你正在使用岭回归，并且你注意到训练误差和验证误差几乎相等且相当高。你是否会说模型存在高偏差或高方差？你应该增加正则化超参数α还是减小它呢？
模型存在高偏差，高偏差是欠拟合，所以要减小正则化超参数α

10. 为什么要使用：a.岭回归而不是简单的线性回归（即没有任何正则化）？b.Lasso而不是岭回归？c.弹性网络而不是Lasso回归？
a.数据存在特征之间高度相关的情况下，简单的线性回归会导致模型的权重估计不稳定，甚至可能无法求解。岭回归通过引入L2正则化，可以有效地解决这一问题。
b.Lasso回归通过L1正则化，可以将一些特征的权重压缩到0，从而实现特征选择。
c.弹性网络同时包含L1和L2正则化项，既可以实现特征选择（L1正则化），又可以防止权重过大（L2正则化）。这使得弹性网络在处理具有多重共线性的高维数据时表现更好。

11. 假设你要将图片分类为室外/室内和白天/夜间。你应该实现两个逻辑回归分类器还是一个softmax回归分类器？
使用一个softmax回归分类器，因为它能够更好地捕捉室外/室内和“白天/夜间”之间的关联。

编程题：

In [183]:
# todo 编程题: 在不使用sklearn的情况下，仅使用Numpy，为softmax回归实现带早停的批量梯度下降，将它用于分类任务，例如鸢尾花数据集

#  注意：
#  1. 要实现l2正则化
#  2. 除了数据读取，其他仅使用numpy，包括训练集+验证集分离，以及softmax预测 和 损失计算

In [1]:
import numpy as np
from keras_core.src.utils.dataset_utils import labels_to_dataset
from sklearn.datasets import load_iris



Using TensorFlow backend


In [30]:
def prepare_data():
    iris = load_iris()
    X = iris.data
    y = iris.target

    X_bias = np.insert(X, 0, 1, axis=1)
    return  X_bias,y

X, y = prepare_data()
print(X.shape, y.shape)


(150, 5) (150,)


In [31]:
def split_data(X, y, test_portion=0.2, valid_portion=0.2):
    num_samples = len(X)
    indices = np.random.permutation(num_samples)

    test_split = int(num_samples * test_portion)
    valid_split = test_split + int(num_samples * valid_portion)

    X_train = X[indices[valid_split:]]
    y_train = y[indices[valid_split:]]
    X_valid = X[indices[test_split:valid_split]]
    y_valid = y[indices[test_split:valid_split]]
    X_test = X[indices[:test_split]]
    y_test = y[indices[:test_split]]

    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [32]:
def encode_labels(y, num_classes=None):
    if num_classes is None:
        num_classes = np.max(y) + 1
    return np.eye(num_classes)[y]

In [ ]:
class SoftmaxRegression:
    def __init__(self, learning_rate=0.1, alpha=0.1, max_epochs=5000, patience=500):
        self.lr = learning_rate
        self.alpha = alpha
        self.max_epochs = max_epochs
        self.patience = patience
        self.weights = None

In [ ]:
def _softmax(self, z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [ ]:
def _compute_loss(self, X, y_encoded):
    logits = X @ self.weights
    probs = self._softmax(logits)
    cross_entropy = -np.mean(np.sum(y_encoded * np.log(probs + 1e-7), axis=1))
    l2_penalty = 0.5 * self.alpha * np.sum(self.weights[1:]**2)
    return cross_entropy + l2_penalty

In [ ]:
def fit(self, X_train, y_train_encoded, X_valid, y_valid_encoded):
    n_features = X_train.shape[1]
    n_classes = y_train_encoded.shape[1]
    self.weights = np.random.randn(n_features, n_classes) * 0.01

    best_loss = float('inf')
    best_weights = None
    no_improvement = 0

    for epoch in range(self.max_epochs):

        logits = X_train @ self.weights
        probs = self._softmax(logits)


        grad = X_train.T @ (probs - y_train_encoded) / len(X_train)
        grad[1:] += self.alpha * self.weights[1:]


        self.weights -= self.lr * grad


        valid_loss = self._compute_loss(X_valid, y_valid_encoded)


        if valid_loss < best_loss:
            best_loss = valid_loss
            best_weights = self.weights.copy()
            no_improvement = 0
        else:
            no_improvement += 1
            if no_improvement >= self.patience:
                print(f"Early stopping at epoch {epoch}")
                self.weights = best_weights
                break

In [ ]:
# 数据划分
X_train, X_valid, X_test, y_train, y_valid, y_test = split_data(X, y)

# 标签编码
y_train_encoded = encode_labels(y_train)
y_valid_encoded = encode_labels(y_valid)
y_test_encoded = encode_labels(y_test)

# 训练模型
model = SoftmaxRegression()
model.fit(X_train, y_train_encoded, X_valid, y_valid_encoded)